In [ ]:
"""A script to auto-backup data."""

def dir_backup(
    src_path: str,
    dst_path: str,
):
    """Backup data from src to dst.
    
    Parameters
    ----------
    src_path: str
        Path to the source files. Could point to one file or one directory.

    dst_path: str
        Path to the backup destination where files will be stored. Could point to one file or one directory.
    """
    pass

